In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Cargar datos
df = pd.read_csv("results/evaluation_results/evaluation_results.csv")

# Crear matriz para el heatmap
pivot_accuracy = df.pivot(index='algorithm', columns='encoding', values='accuracy')

# Crear heatmap interactivo
fig = px.imshow(
    pivot_accuracy.values,
    x=pivot_accuracy.columns,
    y=pivot_accuracy.index,
    color_continuous_scale='RdYlBu_r',
    aspect='auto',
    template='presentation',
    title='Mapa de Calor: Accuracy por Algoritmo y Encoding'
)

# Personalizar el gráfico
fig.update_layout(
    title={
        'text': 'Mapa de Calor: Accuracy por Algoritmo y Encoding',
        'x': 0.5,
        'font': {'size': 24}
    },
    xaxis_title='Técnica de Codificación',
    yaxis_title='Algoritmo',
    font=dict(size=14),
    height=900,
    width=1200
)

# Añadir valores como texto
for i, algorithm in enumerate(pivot_accuracy.index):
    for j, encoding in enumerate(pivot_accuracy.columns):
        value = pivot_accuracy.iloc[i, j]
        if not pd.isna(value):
            fig.add_annotation(
                x=j, y=i,
                text=f'{value:.3f}',
                showarrow=False,
                font=dict(color='white' if value < 0.97 else 'black', size=10)
            )

# Rotar etiquetas del eje x
fig.update_xaxes(tickangle=45)

fig.show()

In [3]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Cargar datos
df = pd.read_csv("results/evaluation_results/evaluation_results.csv")

# Calcular promedios por algoritmo
metrics_cols = ['accuracy', 'f1_score_weighted', 'f1_score_macro', 
                'precision_weighted', 'precision_macro', 'recall_weighted', 'recall_macro']

algo_stats = df.groupby('algorithm')[metrics_cols].mean()

# Normalizar métricas para mejor visualización (0-1)
algo_stats_norm = algo_stats.copy()

# Crear gráfico de radar
algorithms = algo_stats.index.tolist()
metrics_labels = ['Accuracy', 'F1 Weighted', 'F1 Macro', 
                  'Precision Weighted', 'Precision Macro', 
                  'Recall Weighted', 'Recall Macro']

colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

fig = go.Figure()

# Añadir cada algoritmo
for i, algo in enumerate(algorithms):
    values = algo_stats_norm.loc[algo].tolist()
    values += [values[0]]  # Cerrar el polígono
    
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=metrics_labels + [metrics_labels[0]],
        fill='toself',
        name=algo,
        line_color=colors[i],
        fillcolor=colors[i],
        opacity=0.6
    ))

# Personalizar layout
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0.92, 1.0],  # Rango específico para mejor visibilidad
            tickvals=[0.92, 0.94, 0.96, 0.98, 1.0],
            ticktext=['0.92', '0.94', '0.96', '0.98', '1.0']
        )),
    showlegend=True,
    title={
        'text': 'Comparación Multimétrica por Algoritmo (Radar)',
        'x': 0.5,
        'font': {'size': 24}
    },
    template='presentation',
    height=700,
    width=800,
    font=dict(size=14)
)

fig.show()

In [5]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar datos detallados de tiempos
df_times = pd.read_csv('results/training_times/training_times_complete_detailed.csv')

# Crear categorías de encoding
df_times['encoding_type'] = df_times['encoding'].apply(
    lambda x: 'AS' if x.startswith('AS_') else 'PS'
)

# Gráfico de violín con puntos
fig = px.violin(
    df_times,
    x='modelo',
    y='tiempo',
    color='encoding_type',
    box=True,
    points='all',
    hover_data=['encoding', 'iter'],
    template='presentation',
    title='Distribución de Tiempos de Entrenamiento por Algoritmo',
    labels={
        'tiempo': 'Tiempo de Entrenamiento (segundos)',
        'modelo': 'Algoritmo',
        'encoding_type': 'Tipo de Encoding'
    }
)

# Personalizar layout
fig.update_layout(
    title={
        'text': 'Distribución de Tiempos de Entrenamiento por Algoritmo',
        'x': 0.5,
        'font': {'size': 24}
    },
    xaxis_title='Algoritmo',
    yaxis_title='Tiempo de Entrenamiento (segundos)',
    yaxis_type='log',  # Escala logarítmica para mejor visualización
    font=dict(size=14),
    height=600,
    width=1000,
    showlegend=True
)

# Añadir líneas de referencia para percentiles
for modelo in df_times['modelo'].unique():
    data_subset = df_times[df_times['modelo'] == modelo]['tiempo']
    p95 = data_subset.quantile(0.95)
    fig.add_hline(y=p95, line_dash="dash", line_color="red", 
                  annotation_text=f"P95: {p95:.1f}s", opacity=0.7)

fig.show()

In [6]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Cargar datos
df_train = pd.read_csv('results/datos_tablas_entrenamiento.csv')
df_eval = pd.read_csv('results/datos_tablas_evaluacion.csv')

# Renombrar para consistencia
df_train = df_train.rename(columns={'model_name': 'algorithm'})
df_eval = df_eval.rename(columns={'model_name': 'algorithm'})

# Combinar datasets
combined = pd.merge(df_train, df_eval, on=['algorithm', 'encoding'], suffixes=('_train', '_test'))

# Filtrar top 10 por accuracy de test
top_combinations = combined.nlargest(10, 'accuracy_test')

# Crear gráfico con subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Accuracy', 'F1-Score Weighted', 'Precision Weighted', 'Recall Weighted'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

metrics = ['accuracy', 'f1_score_weighted', 'precision_weighted', 'recall_weighted']
positions = [(1,1), (1,2), (2,1), (2,2)]

colors_train = '#3498db'
colors_test = '#e74c3c'

for i, metric in enumerate(metrics):
    row, col = positions[i]
    
    # Añadir barras de entrenamiento
    fig.add_trace(
        go.Bar(
            name=f'{metric.title()} Train' if i == 0 else '',
            x=[f"{row['algorithm']}<br>{row['encoding'][:15]}{'...' if len(row['encoding']) > 15 else ''}" 
               for _, row in top_combinations.iterrows()],
            y=top_combinations[f'{metric}_train'].values,
            marker_color=colors_train,
            opacity=0.7,
            showlegend=True if i == 0 else False,
            legendgroup='train'
        ),
        row=row, col=col
    )
    
    # Añadir barras de test
    fig.add_trace(
        go.Bar(
            name=f'{metric.title()} Test' if i == 0 else '',
            x=[f"{row['algorithm']}<br>{row['encoding'][:15]}{'...' if len(row['encoding']) > 15 else ''}" 
               for _, row in top_combinations.iterrows()],
            y=top_combinations[f'{metric}_test'].values,
            marker_color=colors_test,
            opacity=0.7,
            showlegend=True if i == 0 else False,
            legendgroup='test'
        ),
        row=row, col=col
    )

# Actualizar layout
fig.update_layout(
    title={
        'text': 'Comparación Rendimiento: Entrenamiento vs Evaluación (Top 10)',
        'x': 0.5,
        'font': {'size': 24}
    },
    template='presentation',
    height=800,
    width=1400,
    barmode='group',
    font=dict(size=12),
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Rotar etiquetas del eje x
fig.update_xaxes(tickangle=45)

# Establecer rango del eje y para mejor visualización
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_yaxes(range=[0.90, 1.0], row=i, col=j)

fig.show()

In [7]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Cargar datos
df_eval = pd.read_csv('results/evaluation_results/evaluation_results.csv')
df_times = pd.read_csv('results/training_times/training_times_complete_detailed.csv')
df_memory = pd.read_csv('datos/resultados_codificacion_tiempo_memoria.csv')

# Crear dashboard con múltiples subplots
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=(
        'Top 10 Accuracy por Algoritmo',
        'Distribución Tiempo por Encoding Type', 
        'Memoria vs Accuracy',
        'F1-Score Weighted por Algoritmo',
        'Correlación Métricas de Rendimiento',
        'Efficiency Score Distribution'
    ),
    specs=[
        [{"type": "bar"}, {"type": "box"}, {"type": "scatter"}],
        [{"type": "bar"}, {"type": "scatter"}, {"type": "histogram"}]
    ],
    horizontal_spacing=0.12,
    vertical_spacing=0.15
)

# 1. Top 10 Accuracy por Algoritmo (Barras)
top_10 = df_eval.nlargest(10, 'accuracy')
colors_algo = {'SVM': '#1f77b4', 'RandomForest': '#ff7f0e', 'XGBoost': '#2ca02c'}

for algo in top_10['algorithm'].unique():
    algo_data = top_10[top_10['algorithm'] == algo]
    fig.add_trace(
        go.Bar(
            x=algo_data['encoding'].apply(lambda x: x[:12] + '...' if len(x) > 12 else x),
            y=algo_data['accuracy'],
            name=algo,
            marker_color=colors_algo[algo],
            showlegend=True,
            legendgroup=f'algo_{algo}'
        ),
        row=1, col=1
    )

# 2. Distribución de tiempo por tipo de encoding (Box plot)
df_times['encoding_type'] = df_times['encoding'].apply(lambda x: 'AS' if x.startswith('AS_') else 'PS')
for enc_type in df_times['encoding_type'].unique():
    data = df_times[df_times['encoding_type'] == enc_type]
    fig.add_trace(
        go.Box(
            y=data['tiempo'],
            name=enc_type,
            boxpoints='outliers',
            showlegend=False
        ),
        row=1, col=2
    )

# 3. Memoria vs Accuracy (Scatter)
fig.add_trace(
    go.Scatter(
        x=df_eval['input_data_mb'],
        y=df_eval['accuracy'],
        mode='markers',
        marker=dict(
            size=8,
            color=df_eval['f1_score_weighted'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="F1-Score", x=0.65, len=0.4)
        ),
        text=df_eval['algorithm'],
        showlegend=False
    ),
    row=1, col=3
)

# 4. F1-Score Weighted por Algoritmo (Barras horizontales)
f1_by_algo = df_eval.groupby('algorithm')['f1_score_weighted'].mean().sort_values(ascending=True)
fig.add_trace(
    go.Bar(
        x=f1_by_algo.values,
        y=f1_by_algo.index,
        orientation='h',
        marker_color=['#1f77b4', '#ff7f0e', '#2ca02c'],
        showlegend=False
    ),
    row=2, col=1
)

# 5. Correlación Precision vs Recall (Scatter)
fig.add_trace(
    go.Scatter(
        x=df_eval['precision_weighted'],
        y=df_eval['recall_weighted'],
        mode='markers',
        marker=dict(
            size=10,
            color=df_eval['algorithm'].map({'SVM': 0, 'RandomForest': 1, 'XGBoost': 2}),
            colorscale=[[0, '#1f77b4'], [0.5, '#ff7f0e'], [1, '#2ca02c']],
            line=dict(width=1, color='black')
        ),
        text=df_eval['encoding'],
        showlegend=False
    ),
    row=2, col=2
)

# 6. Distribución de Efficiency Score (Histograma)
fig.add_trace(
    go.Histogram(
        x=df_eval['efficiency_score'],
        nbinsx=20,
        marker_color='lightblue',
        marker_line_color='black',
        marker_line_width=1,
        showlegend=False
    ),
    row=2, col=3
)

# Personalizar layout general
fig.update_layout(
    title={
        'text': 'Dashboard Integral: Análisis Comparativo de Rendimiento',
        'x': 0.5,
        'font': {'size': 28}
    },
    template='presentation',
    height=900,
    width=1600,
    font=dict(size=11),
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.1,
        xanchor="center",
        x=0.5
    )
)

# Personalizar ejes específicos
fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_yaxes(type='log', row=1, col=2)
fig.update_xaxes(title_text="Memoria (MB)", row=1, col=3)
fig.update_yaxes(title_text="Accuracy", row=1, col=3)
fig.update_xaxes(title_text="F1-Score Weighted Promedio", row=2, col=1)
fig.update_xaxes(title_text="Precision Weighted", row=2, col=2)
fig.update_yaxes(title_text="Recall Weighted", row=2, col=2)
fig.update_xaxes(title_text="Efficiency Score", row=2, col=3)
fig.update_yaxes(title_text="Frecuencia", row=2, col=3)

# Añadir líneas de referencia donde sea apropiado
fig.add_hline(y=0.95, line_dash="dash", line_color="red", opacity=0.5, row=1, col=3)
fig.add_vline(x=0.95, line_dash="dash", line_color="red", opacity=0.5, row=2, col=2)
fig.add_hline(y=0.95, line_dash="dash", line_color="red", opacity=0.5, row=2, col=2)

fig.show()